In [168]:
import requests
import pandas as pd
import numpy as np
import json
import os

class ConnectAPI:

    def __init__(self):
        self.api_token = 'test85g57'
        self.season_id = '2012'
        self.matches= "https://api.football-data-api.com/league-matches?key={api_token}&season_id={season_id}"
        self.team = "https://api.football-data-api.com/league-teams?key={api_token}&season_id={season_id}&include=stats"
        self.current_path = os.getcwd().partition('notebooks')[0]
        
    def get_json(self, url:str, **kwargs) -> json: 
        url = url.format(api_token = self.api_token, **kwargs)
        self.response = requests.get(url)
        print(self.response.status_code,'status_code')
        print(self.response.request,'request')
        jObject = self.response.json()
        return jObject
    
class MyFunctions:

    def open_json(project_path:str, relative_path:str, file_name:str):
        jsobject = open(project_path + relative_path + "\\" + file_name, 'r')
        json_file = json.load(jsobject)
        return json_file
    
class TableCreating:
    
    def create_table(df:pd.DataFrame, home_away:str) -> pd.DataFrame:
        if home_away == 'home':
            goals_scored = ('homeGoalCount', 'sum')
            goals_concened= ('awayGoalCount', 'sum')
        elif home_away == 'away':
            goals_scored = ('awayGoalCount', 'sum')
            goals_concened= ('homeGoalCount', 'sum')
            
        df = df.groupby([f'{home_away}_team', f'{home_away}ID'] ).agg(
        matches_played = (f'{home_away}_team', 'count'),
        goals_scored = goals_scored,
        goals_concened= goals_concened,    
        ).reset_index()
        
        return df
    
    def create_result_table(df, home_away:str) -> pd.DataFrame:
        
        df = df[[f'{home_away}ID',f'{home_away}_team', 'result']]
        pivot_df = pd.pivot_table(df, index=[f'{home_away}ID', f'{home_away}_team'], columns='result', aggfunc=len, fill_value=0).reset_index() #kaŽkodėl čia len sumuoja :\
        
        if home_away == 'home': df = pivot_df[['homeID','home_team', 'home_win', 'draw', 'away_win']].rename(
            columns={
                'homeID':'teamID',
                'home_team':'team',
                'home_win':'win',
                "away_win": "losse"})
        elif home_away == 'away': df = pivot_df[['awayID','away_team', 'away_win', 'draw', 'home_win']].rename(
            columns={
                'awayID':'teamID',
                'away_team':'team',
                'away_win':'win',
                "home_win": "losse"})
            
        return df
        
class Tables(ConnectAPI):
    
    def __init__(self):
        super().__init__()
        self.create_teams_table()

    def create_teams_table(self) -> pd.DataFrame:
        jsondb_teams = self.get_json(url = self.team, season_id = self.season_id)
        global TEAMS_df
        TEAMS_df = pd.DataFrame(jsondb_teams['data'])
        
        
    def create_my_table(self, json_object:json, data:str = 'data', columns:list = []) -> pd.DataFrame:
        """data default: 'data' 
           column default: empty list"""
        df = pd.DataFrame(json_object[data])
        if columns == []: 
            return df
        else: 
            return df[columns]
        
    def create_general_table(self) -> pd.DataFrame:
        
        json_columns = MyFunctions.open_json(project_path=dt.current_path, relative_path="json", file_name='tables_headers.json')
        columns = json_columns['tables']['general_table']['columns']
        jsondb_matches = dt.get_json(url = dt.matches, season_id = dt.season_id)
        df = self.create_my_table(json_object=jsondb_matches, columns=columns)
        
        matches = pd.merge(df, TEAMS_df[['id','name']], how = 'left', left_on='homeID', right_on='id') 
        matches2 = pd.merge(matches, TEAMS_df[['id','name']], how = 'left', left_on='awayID', right_on='id' ) 
        df = matches2.drop(columns=['id', 'id_y']).rename(columns={"id_x":"id", 'name_x':'home_team','name_y':'away_team'}) 
    
        df = df[['id', 'home_team', 'away_team', 'season', 'status', 'roundID','game_week', 'homeGoals', 
               'awayGoals', 'homeGoalCount', 'awayGoalCount', 'totalGoalCount', 'homeID', 'awayID']]
        
        df['result'] = np.where(df['homeGoals'] > df['awayGoals'], 'home_win', 
                        np.where(df['homeGoals'] < df['awayGoals'], 'away_win', 'draw'))
        
        return df
        
        
    def create_home_table(self, df:pd.DataFrame) -> pd.DataFrame:
        df = TableCreating.create_table(df, 'home')
        
        
        
        return df
    
        
        
        
        
class Calculate:
    
    def __init__(self):
        pass

    
if __name__ == "__main__":
    
    dt = Tables()
    matches = dt.create_general_table()
    home = dt.create_home_table(matches)
    df = TableCreating.create_result_table(matches, 'home')

    


200 status_code
<PreparedRequest [GET]> request
200 status_code
<PreparedRequest [GET]> request


In [169]:
df

result,teamID,team,win,draw,losse
0,59,Arsenal FC,12,0,7
1,92,Tottenham Hotspur FC,9,0,10
2,93,Manchester City FC,15,0,4
3,108,Leicester City FC,11,2,6
4,143,Crystal Palace FC,10,1,8
5,144,Everton FC,9,2,8
6,145,Burnley FC,13,1,5
7,146,Southampton FC,12,0,7
8,148,AFC Bournemouth,7,2,10
9,149,Manchester United FC,14,1,4
